***Importa biblioteca de analise de dados***

In [1]:
import pandas as pd

***Informa o local do arquivo de analise de dados do arkmeds e o transforma em um Dataframe***

In [2]:
# Defina aqui o nome do arquivo do mês a ser processado
caminho_arquivo = 'C:/Users/Rafael Ribeiro/Documents/OS/ordens_servico_janeiro.xls'

# Carregar os dados
df = pd.read_excel(caminho_arquivo)
print(f"Arquivo carregado: {caminho_arquivo}")
print(f"Total de registros: {len(df)}")

Arquivo carregado: C:/Users/Rafael Ribeiro/Documents/OS/ordens_servico_janeiro.xls
Total de registros: 701


***Informa quantas ordens de serviço corretivas foram abertas no total (engenharia clinica e predial)***

In [3]:
corretivas = df[df['TIPO SERVIÇO'] == 'Manutenção Corretiva']
total_corretivas_abertas = corretivas.shape[0]
print(total_corretivas_abertas)

220


***Informa quantas ordens de serviço corretivas foram fechadas (Engenharia clinica e predial)***

In [4]:
corretivas_fechadas = corretivas[corretivas['ESTADO'] == 'Fechada']
total_corretivas_fechadas = corretivas_fechadas.shape[0]
print(total_corretivas_fechadas)

217


***Informa quantas ordens de serviço preventivas foram abertas no total (engenharia clinica e predial)***

In [5]:
preventivas = df[df['TIPO SERVIÇO'] == 'Manutenção Preventiva']
total_preventivas_abertas = preventivas.shape[0]
print(total_preventivas_abertas)

452


***Informa quantas ordens de serviço preventivas foram fechadas (Engenharia Clinica e Predial)***

In [6]:
preventivas_fechadas =  preventivas[preventivas['ESTADO'] == 'Fechada']
total_preventivas_fechadas = preventivas_fechadas.shape[0]
print(total_preventivas_fechadas)

299


***Filtra pelo quadro de engenharia clinica***

In [7]:
engenharia_clinica = df[df['QUADRO DE TRABALHO'] == 'Engenharia Clínica']

***Total de corretivas que foram criadas para engenharia clinica***

In [8]:
corretivas_engenharia = engenharia_clinica[engenharia_clinica['TIPO SERVIÇO'] == 'Manutenção Corretiva']
total_corretivas_criadas_engenharia = corretivas_engenharia.shape[0]
print(f"Total de manutenções corretivas abertas no mes para engenharia: {total_corretivas_criadas_engenharia}")

Total de manutenções corretivas abertas no mes para engenharia: 72


***Total de corretivas fechadas pela engenharia clinica***

In [9]:
corretivas_engenharia_fechadas = corretivas_engenharia[corretivas_engenharia['ESTADO'] == 'Fechada']
total_corretivas_engenharia_fechadas = corretivas_engenharia_fechadas.shape[0]
print(f"Total de manutenções corretivas fechadas no mes: {total_corretivas_engenharia_fechadas}")

Total de manutenções corretivas fechadas no mes: 72


In [10]:
percentual_corretivas_fechadas_engenharia = (total_corretivas_engenharia_fechadas / total_corretivas_criadas_engenharia) * 100
print(f"Percentual de ordens corretivas fechadas: {percentual_corretivas_fechadas_engenharia:.2f}%")

Percentual de ordens corretivas fechadas: 100.00%


***INDICADOR COMG - Taxa de atendimento manutenção corretivas dentro do prazo***

In [11]:
corretivas_engenharia_regular = corretivas_engenharia[corretivas_engenharia['Estado tempo atendimento'] == 'Regular']
total_corretivas_engenharia_fechadas_regular = corretivas_engenharia_regular.shape[0]
print(f"Total de manutenções corretivas atendidas dentro do prazo! {total_corretivas_engenharia_fechadas_regular}")

Total de manutenções corretivas atendidas dentro do prazo! 53


In [12]:
percentual_corretivas_fechadas_engenharia_regular = (total_corretivas_engenharia_fechadas_regular/total_corretivas_criadas_engenharia) * 100
print(f"Percentual de ordens corretivas Atendidas dentro do prazo: {percentual_corretivas_fechadas_engenharia_regular:.2f}%")

Percentual de ordens corretivas Atendidas dentro do prazo: 73.61%


***Total de preventivas abertas - engenharia clinica***

In [13]:
preventivas_engenharia = engenharia_clinica[engenharia_clinica['TIPO SERVIÇO'] == 'Manutenção Preventiva']
total_preventivas_geradas_predial = preventivas_engenharia.shape[0]
print(f"Total de manutenções preventivas abertas para engenharia no mes: {total_preventivas_geradas_predial}")

Total de manutenções preventivas abertas para engenharia no mes: 116


***Total de preventivas fechadas - engenharia clinica***

In [14]:
preventivas_engenharia_fechadas = preventivas_engenharia[preventivas_engenharia['ESTADO'] != 'Aberta']
total_preventivas_engenharia_fechadas = preventivas_engenharia_fechadas.shape[0]
print(f"Total de manutenções preventivas fechadas pela engenharia no mes: {total_preventivas_engenharia_fechadas}")

Total de manutenções preventivas fechadas pela engenharia no mes: 116


***Percentual de preventivas fechadas - engenharia clinica***

In [15]:
percentual_preventivas_fechadas_engenharia= (total_preventivas_engenharia_fechadas / total_preventivas_geradas_predial) * 100
print(f"Percentual de ordens preventivas fechadas: {percentual_preventivas_fechadas_engenharia:.2f}%")

Percentual de ordens preventivas fechadas: 100.00%


***SLA triagem emergente engenharia clinica***

In [16]:
sla_triagem_emergente_engenharia =  corretivas_engenharia[corretivas_engenharia['PRIORIDADE'] == 'Emergente']
sla_triagem_emergente_total_engenharia = sla_triagem_emergente_engenharia.shape[0]
print(f"Total de ordens de serviço emergentes abertas para engenharia clinica: {sla_triagem_emergente_total_engenharia}")

Total de ordens de serviço emergentes abertas para engenharia clinica: 2


***Sla triagem emergente engenharia clinica %***

In [17]:
sla_triagem_emergente_regular_engenharia = sla_triagem_emergente_engenharia[sla_triagem_emergente_engenharia['Estado tempo atendimento'] == 'Regular']
sla_triagem_emergente_regular_total_engenharia = sla_triagem_emergente_regular_engenharia.shape[0]
if sla_triagem_emergente_regular_total_engenharia == 0:
    print(f"Não houve chamados emergentes aberto no mes!")
else:
    percentual_sla_triagem_emergente_engenharia = (sla_triagem_emergente_regular_total_engenharia/sla_triagem_emergente_total_engenharia) * 100
    print(f"Total de atendimento emergente da engenharia clinica dentro do prazo: {percentual_sla_triagem_emergente_engenharia}%")

Não houve chamados emergentes aberto no mes!


***SLA fechamento emergente engenharia clinica %***

In [18]:
sla_fechamento_emergente_regular_engenharia = sla_triagem_emergente_engenharia[sla_triagem_emergente_engenharia['Estado tempo fechamento'] == 'Regular']
sla_fechamento_emergente_regular_total_engenharia = sla_fechamento_emergente_regular_engenharia.shape[0]
if sla_fechamento_emergente_regular_total_engenharia == 0:
    print("Não houve chamados emergentes aberto no mes!")
else:
    percentual_sla_fechamento_emergente_engenharia = (sla_fechamento_emergente_regular_total_engenharia/sla_triagem_emergente_total_engenharia) * 100
    print(f"Total de ordens serviços emergentes fechadas dentro do prazo {percentual_sla_fechamento_emergente_engenharia}")

Não houve chamados emergentes aberto no mes!


***Sla triagem urgente engenharia clinica***

In [19]:
sla_triagem_urgente_engenharia = corretivas_engenharia[corretivas_engenharia['PRIORIDADE'] == 'Urgente']
sla_triagem_urgente_total_engenharia = sla_triagem_urgente_engenharia.shape[0]
print(f"Total de ordens de serviço urgentes abertas para engenharia clinica {sla_triagem_urgente_total_engenharia}")

Total de ordens de serviço urgentes abertas para engenharia clinica 41


***Sla triagem urgente engenharia %***

In [20]:
sla_triagem_urgente_regular_engenharia = sla_triagem_urgente_engenharia[sla_triagem_urgente_engenharia['Estado tempo atendimento'] == 'Regular']
sla_triagem_urgente_regular_total_engenharia = sla_triagem_urgente_regular_engenharia.shape[0]
if sla_triagem_urgente_total_engenharia == 0:
    print("Não foram abertos chamados de triagem urgente")
else:
    percentual_sla_triagem_urgente_engenharia = (sla_triagem_urgente_regular_total_engenharia / sla_triagem_urgente_total_engenharia) * 100
    print(f"Total de atendimento urgente da engenharia clinica dentro do prazo: {percentual_sla_triagem_urgente_engenharia}%")

Total de atendimento urgente da engenharia clinica dentro do prazo: 92.6829268292683%


***Sla fechamento urgente engenharia %***

In [21]:
sla_fechamento_urgente_regular_engenharia = sla_triagem_urgente_engenharia [sla_triagem_urgente_engenharia ['Estado tempo fechamento'] == 'Regular']
sla_fechamento_urgente_regular_total_engenharia = sla_fechamento_urgente_regular_engenharia.shape[0]
if sla_triagem_urgente_total_engenharia == 0:
    print("Não foram abertos chamados de triagem urgente")
else:
    percentual_sla_fechamento_urgente_regular_total_engenharia = (sla_fechamento_urgente_regular_total_engenharia /sla_triagem_urgente_total_engenharia ) * 100
    print(f"Total de ordens serviços urgentes fechadas dentro do prazo {percentual_sla_fechamento_urgente_regular_total_engenharia}% {sla_fechamento_urgente_regular_total_engenharia}")

Total de ordens serviços urgentes fechadas dentro do prazo 65.85365853658537% 27


***SLA triagem pouco urgente engenharia clinica***

In [22]:
sla_triagem_poucourgente_engenharia = corretivas_engenharia[
    (corretivas_engenharia['PRIORIDADE'] == 'Pouco urgente') | 
    (corretivas_engenharia['PRIORIDADE'] == 'Não urgente')
]
sla_triagem_poucourgente_engenharia_total = sla_triagem_poucourgente_engenharia.shape[0]
print(f"Total de ordens de serviço pouco urgentes abertas para engenharia clínica: {sla_triagem_poucourgente_engenharia_total}")


Total de ordens de serviço pouco urgentes abertas para engenharia clínica: 29


***Sla triagem pouco urgente engenharia %***


In [23]:
sla_triagem_poucourgente_regular_engenharia = sla_triagem_poucourgente_engenharia[sla_triagem_poucourgente_engenharia['Estado tempo atendimento'] == 'Regular']
sla_triagem_poucourgente_regular_engenharia_total = sla_triagem_poucourgente_regular_engenharia.shape[0]
if sla_triagem_poucourgente_engenharia_total == 0:
    print("Não foram abertos chamados de triagem pouco urgente")
else:
    percentual_sla_poucourgente_engenharia = (sla_triagem_poucourgente_regular_engenharia_total/sla_triagem_poucourgente_engenharia_total) * 100
    print(f"Total de atendimento pouco urgente da engenharia clinica dentro do prazo: {percentual_sla_poucourgente_engenharia}%, {sla_triagem_poucourgente_regular_engenharia_total}")

Total de atendimento pouco urgente da engenharia clinica dentro do prazo: 51.724137931034484%, 15


***Sla fechamento pouco urgente engenharia***

In [24]:
sla_fechamento_poucourgente_regular_engenharia = sla_triagem_poucourgente_engenharia[sla_triagem_poucourgente_engenharia['Estado tempo fechamento'] == 'Regular']
sla_fechamento_poucourgente_regular_total_engenharia = sla_fechamento_poucourgente_regular_engenharia.shape[0]
if sla_triagem_poucourgente_engenharia_total == 0:
    print("Não foram abertos chamados de triagem pouco urgente")
else:
    percentual_sla_fechamento_poucourgente_regular_total_engenharia = (sla_fechamento_poucourgente_regular_total_engenharia/sla_triagem_poucourgente_engenharia_total) * 100
    print(f"Total de ordens serviços urgentes fechadas dentro do prazo {percentual_sla_fechamento_poucourgente_regular_total_engenharia} %, {sla_fechamento_poucourgente_regular_total_engenharia}")

Total de ordens serviços urgentes fechadas dentro do prazo 31.03448275862069 %, 9


***Porcentagem de ordem de serviço corretiva emergente***

In [25]:
 Percentagem_emergente = (sla_triagem_emergente_total_engenharia/total_corretivas_criadas_engenharia) * 100
 print(f"Dentro de todas as corretivas {Percentagem_emergente}% foi o total de emergente")

Dentro de todas as corretivas 2.7777777777777777% foi o total de emergente


***Porcentagem de ordem de serviço corretiva urgente***

In [26]:
 percentagem_urgente = (sla_triagem_urgente_total_engenharia/total_corretivas_criadas_engenharia) * 100
 print(f"Dentro de todas as corretivas {percentagem_urgente}% foi o total de emergente")

Dentro de todas as corretivas 56.94444444444444% foi o total de emergente


***Porcentagem de ordem de serviço corretiva pouco urgente***

In [27]:
percentagem_poucourgente = (sla_triagem_poucourgente_engenharia_total/total_corretivas_criadas_engenharia) *100
print(f"Dentro de todas as corretivas {percentagem_poucourgente}% foi o total de emergente")

Dentro de todas as corretivas 40.27777777777778% foi o total de emergente


***Filtra pelo quadro de manutenção predial***

In [28]:
predial = df[df['QUADRO DE TRABALHO'] == 'Predial']

***Total de corretivas abertas para manutenção predial***

In [29]:
corretivas_predial = predial[predial['TIPO SERVIÇO'] == 'Manutenção Corretiva']
total_corretivas_criadas_predial = corretivas_predial.shape[0]
print(f"Total de manutenções corretivas abertas no mes para predial:{total_corretivas_criadas_predial}")


Total de manutenções corretivas abertas no mes para predial:146


***Total de corretivas fechadas pela predial***

In [30]:
corretivas_predial_fechadas = corretivas_predial[corretivas_predial['ESTADO'] == 'Fechada']
total_corretivas_predial_fechadas = corretivas_predial_fechadas.shape[0]
print(f"Total de manutenções corretivas fechadas pela predial: {total_corretivas_predial_fechadas}")

Total de manutenções corretivas fechadas pela predial: 144


***Percentual de corretivas fechadas - predial***

In [31]:
percentual_corretivas_fechadas_predial = (total_corretivas_predial_fechadas / total_corretivas_criadas_predial) * 100
print(f"Percentual de ordens corretivas fechadas pela predial: {percentual_corretivas_fechadas_predial:.2f}%")

Percentual de ordens corretivas fechadas pela predial: 98.63%


***Total de preventivas abertas - predial***

In [32]:
preventivas_predial = predial[predial['TIPO SERVIÇO'] == 'Manutenção Preventiva']
total_preventivas_geradas_predial = preventivas_predial.shape[0]
print(f"Total de manutenções preventivas geradas para predial no mes: {total_preventivas_geradas_predial}")

Total de manutenções preventivas geradas para predial no mes: 321


***Total de preventivas Fechadas - predial***

In [33]:
preventiva_predial_fechada = preventivas_predial[preventivas_predial['ESTADO'] != 'Aberta']
total_preventivas_predial_fechada = preventiva_predial_fechada.shape[0]
print(f"Total de manutenções preventivas fechadas pela predial no mes: {total_preventivas_predial_fechada}")

Total de manutenções preventivas fechadas pela predial no mes: 319


 ***Preventivas de ar***

In [34]:
preventivas_arcondicionado = preventivas_predial[preventivas_predial['PLANO'] == 'Preventiva Mensal Refrigeração']
total_preventivas_arcondicionado = preventivas_arcondicionado.shape[0]
print(f'Total de preventivas abertas de ar condicionado no mes: {total_preventivas_arcondicionado}')

Total de preventivas abertas de ar condicionado no mes: 0


***Preventivas de ar realizadas***

In [35]:
preventivas_arcondicionado_fechadas = preventivas_arcondicionado[preventivas_arcondicionado['ESTADO'] != 'Aberta']
total_preventivas_arcondicionado_fechadas = preventivas_arcondicionado_fechadas.shape[0]
print(f"total de preventivas feitas de ar condicionado no mes {total_preventivas_arcondicionado_fechadas}")
                                                                                             

total de preventivas feitas de ar condicionado no mes 0


***Demais preventivas predial***

In [36]:
preventivas_demais = preventivas_predial[preventivas_predial['PLANO'] != 'Preventiva Mensal Refrigeração']
total_preventivas_demais = preventivas_demais.shape[0]
print(f'Total de preventivas abertas no mes: {total_preventivas_demais}')

Total de preventivas abertas no mes: 321


***Demais preventivas predial fechadas***

In [37]:
preventivas_demais_fechadas = preventivas_demais[preventivas_demais['ESTADO'] != 'Aberta']
total_preventivas_demais_fechadas = preventivas_demais_fechadas.shape[0]
print(f"total de preventivas feitas de ar condicionado no mes {total_preventivas_demais_fechadas}")

total de preventivas feitas de ar condicionado no mes 319


***Percentual de preventivas fechadas - Predial***

In [38]:
percentual_preventivas_fechadas_predial = (total_preventivas_predial_fechada / total_preventivas_geradas_predial) * 100
print(f"Percentual de ordens preventivas fechadas pela predial: {percentual_preventivas_fechadas_predial:.2f}%")

Percentual de ordens preventivas fechadas pela predial: 99.38%


***SLA triagem emergente - Predial***

In [39]:
sla_triagem_emergente_predial = corretivas_predial[corretivas_predial['PRIORIDADE'] == 'Emergente']
sla_triagem_emergente_total_predial = sla_triagem_emergente_predial.shape[0]
print(f"Total de ordens de serviço emergentes abertas para predial: {sla_triagem_emergente_total_predial}")

Total de ordens de serviço emergentes abertas para predial: 19


***SLA triagem emergente Predial %***

In [40]:
sla_triagem_emergente_regular_predial = sla_triagem_emergente_predial[sla_triagem_emergente_predial['Estado tempo atendimento'] == 'Regular']
sla_triagem_emergente_regular_total_predial = sla_triagem_emergente_regular_predial.shape[0]
if sla_triagem_emergente_regular_total_predial == 0:
    sla_triagem_emergente_regular_total_predial = 0
    print(f"Não houve chamados emergente aberto no mes! {sla_triagem_emergente_regular_total_predial}")
else:
    percentual_sla_triagem_emergente_predial = (sla_triagem_emergente_regular_total_predial / sla_triagem_emergente_total_predial) * 100
    print(f"Total de atendimento emergente da predial dentro do prazo:{percentual_sla_triagem_emergente_predial}")

Total de atendimento emergente da predial dentro do prazo:52.63157894736842


***SLA fechamento emergente Predial %***

In [41]:
sla_fechamento_emergente_regular_predial = sla_triagem_emergente_predial[sla_triagem_emergente_predial['Estado tempo fechamento'] == 'Regular']
sla_fechamento_emergente_regular_total_predial = sla_fechamento_emergente_regular_predial.shape[0]
if sla_fechamento_emergente_regular_total_predial == 0:
    print("Não houve chamados emergentes aberto no mes!")
else:
    percentual_sla_fechamento_emergente_predial = (sla_fechamento_emergente_regular_total_predial/sla_triagem_emergente_total_predial) * 100
    print(f"Total de ordens serviços emergentes fechadas dentro do prazo {percentual_sla_fechamento_emergente_predial}")

Total de ordens serviços emergentes fechadas dentro do prazo 52.63157894736842


***SLA triagem urgente Predial***

In [42]:
sla_triagem_urgente_predial = corretivas_predial[corretivas_predial['PRIORIDADE'] == 'Urgente']
sla_triagem_urgente_predial_total = sla_triagem_urgente_predial.shape[0]
print(f"Total de ordens de serviço urgentes abertas para engenharia clinica {sla_triagem_urgente_predial_total}")

Total de ordens de serviço urgentes abertas para engenharia clinica 45


***Percentual de corretivas fechadas -  engenharia clinica***

***SLA triagem urgente Predial %***

In [43]:
sla_triagem_urgente_regular_predial = sla_triagem_urgente_predial[sla_triagem_urgente_predial['Estado tempo atendimento'] == 'Regular']
sla_triagem_urgente_regular_total_predial = sla_triagem_urgente_regular_predial.shape[0]
percentual_sla_triagem_urgente_predial = (sla_triagem_urgente_regular_total_predial / sla_triagem_urgente_predial_total) * 100
print(f"Total de atendimento urgente da engenharia clinica dentro do prazo: {percentual_sla_triagem_urgente_predial}%")

Total de atendimento urgente da engenharia clinica dentro do prazo: 91.11111111111111%


***SLA fechamento urgente Predial %***

In [44]:
sla_fechamento_urgente_regular_predial = sla_triagem_urgente_predial [sla_triagem_urgente_predial ['Estado tempo fechamento'] == 'Regular']
sla_fechamento_urgente_regular_total_predial = sla_fechamento_urgente_regular_predial.shape[0]
percentual_sla_fechamento_urgente_regular_total_predial = (sla_fechamento_urgente_regular_total_predial / sla_triagem_urgente_predial_total ) * 100
print(f"Total de ordens serviços urgentes fechadas dentro do prazo {percentual_sla_fechamento_urgente_regular_total_predial}%")

Total de ordens serviços urgentes fechadas dentro do prazo 51.11111111111111%


***SLA triagem pouco urgente Predial***

In [45]:
sla_triagem_poucourgente_predial = corretivas_predial[
    (corretivas_predial['PRIORIDADE'] == 'Pouco urgente') 
    ]
sla_triagem_poucourgente_predial_total = sla_triagem_poucourgente_predial.shape[0]
print(f"Total de ordens de serviço pouco urgentes abertas para engenharia clinica: {sla_triagem_poucourgente_predial_total}")

Total de ordens de serviço pouco urgentes abertas para engenharia clinica: 77


***SLA triagem pouco urgente Predial %***

In [46]:
sla_triagem_poucourgente_regular_predial = sla_triagem_poucourgente_predial[sla_triagem_poucourgente_predial['Estado tempo atendimento'] == 'Regular']
sla_triagem_poucourgente_regular_predial_total = sla_triagem_poucourgente_regular_predial.shape[0]
percentual_sla_poucourgente_predial = (sla_triagem_poucourgente_regular_predial_total/sla_triagem_poucourgente_predial_total) * 100
print(f"Total de atendimento pouco urgente da engenharia clinica dentro do prazo: {percentual_sla_poucourgente_predial}% , {sla_triagem_poucourgente_regular_predial_total}")

Total de atendimento pouco urgente da engenharia clinica dentro do prazo: 87.01298701298701% , 67


***SLA fechamento pouco urgente Predial***

In [47]:
sla_fechamento_poucourgente_regular_predial = sla_triagem_poucourgente_predial[sla_triagem_poucourgente_predial['Estado tempo fechamento'] == 'Regular']
sla_fechamento_poucourgente_regular_total_predial = sla_fechamento_poucourgente_regular_predial.shape[0]
percentual_sla_fechamento_poucourgente_regular_total_predial = (sla_fechamento_poucourgente_regular_total_predial/sla_triagem_poucourgente_predial_total) * 100
print(f"Total de ordens serviços urgentes fechadas dentro do prazo {percentual_sla_fechamento_poucourgente_regular_total_predial}%, {sla_fechamento_poucourgente_regular_total_predial}")

Total de ordens serviços urgentes fechadas dentro do prazo 61.038961038961034%, 47


***Porcentagem de ordem de serviço corretiva emergente***

In [48]:
Percentagem_emergente = (sla_triagem_emergente_total_predial/total_corretivas_criadas_predial) * 100
print(f"Dentro de todas as corretivas {Percentagem_emergente}% foi o total de emergente")

Dentro de todas as corretivas 13.013698630136986% foi o total de emergente


***Porcentagem de ordem de serviço corretiva urgente***

In [49]:
percentagem_urgente = (sla_triagem_urgente_predial_total/total_corretivas_criadas_predial) * 100
print(f"Dentro de todas as corretivas {percentagem_urgente}% foi o total de emergente")

Dentro de todas as corretivas 30.82191780821918% foi o total de emergente


***Porcentagem de ordem de serviço corretiva pouco urgente***

In [50]:
percentagem_poucourgente = (sla_triagem_poucourgente_predial_total/total_corretivas_criadas_predial) *100
print(f"Dentro de todas as corretivas {percentagem_poucourgente}% foi o total de emergente")

Dentro de todas as corretivas 52.73972602739726% foi o total de emergente


***AR CONDICIONADOS PENDENTES DE PREVENTIVA NO MÊS***

In [51]:
# Configurando a exibição do pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.expand_frame_repr', False)  # Evita que as colunas quebrem em múltiplas linhas
pd.set_option('display.max_rows', 100)  # Mostra até 100 linhas (ajuste conforme necessário)

# Filtrando os dados
preventivas_arcondicionado_abertas = preventivas_arcondicionado[preventivas_arcondicionado['ESTADO'] == 'Aberta']

# Mostrando o DataFrame filtrado de maneira completa
print(preventivas_arcondicionado_abertas)

Empty DataFrame
Columns: [ID, NUMERO, TIPO SERVIÇO, ESTADO, SOLICITANTE, LOCALIZAÇÃO, TAG, RESPONSAVEL, DATA DE CRIAÇÃO, DATA DE CONCLUSÃO, PRIORIDADE, PRAZO, PLANO, OBSERVAÇÕES, QUADRO DE TRABALHO, Horário do início do atendimento, Tempo até o início de atendimento (horas:minutos:segundos), Estado tempo atendimento, Tempo total do serviço (horas:minutos:segundos), Estado tempo fechamento, Tempo de atendimento/ Tempo total do serviço (%)]
Index: []


***CONSOLIDAÇÃO DE INDICADORES - EXPORTAÇÃO PARA CSV***

**Como usar:**
1. Para processar outro mês, altere apenas a célula 2 com o novo nome do arquivo
2. Execute todo o notebook (Run All)
3. O CSV será atualizado automaticamente com o novo mês em uma nova linha
4. Os meses serão ordenados cronologicamente no arquivo final

In [52]:
import re
import os

# Extrair o mês do nome do arquivo (funciona com "servico" ou "serviço")
nome_arquivo = os.path.basename(caminho_arquivo)
match = re.search(r'ordens_servi[cç]o_(\w+)\.xls', nome_arquivo)
mes = match.group(1).capitalize() if match else "Desconhecido"

print(f"Mês identificado: {mes}")

Mês identificado: Janeiro


In [53]:
# Criar dicionário com todos os indicadores calculados
indicadores = {
    'Mês': mes,
    
    # Indicadores Gerais
    'Total Corretivas Abertas': total_corretivas_abertas,
    'Total Corretivas Fechadas': total_corretivas_fechadas,
    'Total Preventivas Abertas': total_preventivas_abertas,
    'Total Preventivas Fechadas': total_preventivas_fechadas,
    
    # Engenharia Clínica - Corretivas
    'ENG - Total Corretivas Abertas': total_corretivas_criadas_engenharia,
    'ENG - Total Corretivas Fechadas': total_corretivas_engenharia_fechadas,
    'ENG - % Corretivas Fechadas': round(percentual_corretivas_fechadas_engenharia, 2),
    'ENG - Corretivas Atendidas no Prazo': total_corretivas_engenharia_fechadas_regular,
    'ENG - % Corretivas Atendidas no Prazo': round(percentual_corretivas_fechadas_engenharia_regular, 2),
    
    # Engenharia Clínica - Preventivas
    'ENG - Total Preventivas Abertas': total_preventivas_geradas_predial,
    'ENG - Total Preventivas Fechadas': total_preventivas_engenharia_fechadas,
    'ENG - % Preventivas Fechadas': round(percentual_preventivas_fechadas_engenharia, 2),
    
    # Engenharia Clínica - SLA Emergente
    'ENG - OS Emergentes': sla_triagem_emergente_total_engenharia,
    'ENG - Emergente Atendidas no Prazo': sla_triagem_emergente_regular_total_engenharia,
    'ENG - % SLA Triagem Emergente': round(percentual_sla_triagem_emergente_engenharia, 2) if sla_triagem_emergente_regular_total_engenharia > 0 else 0,
    'ENG - Emergente Fechadas no Prazo': sla_fechamento_emergente_regular_total_engenharia,
    'ENG - % SLA Fechamento Emergente': round(percentual_sla_fechamento_emergente_engenharia, 2) if sla_fechamento_emergente_regular_total_engenharia > 0 else 0,
    
    # Engenharia Clínica - SLA Urgente
    'ENG - OS Urgentes': sla_triagem_urgente_total_engenharia,
    'ENG - Urgente Atendidas no Prazo': sla_triagem_urgente_regular_total_engenharia,
    'ENG - % SLA Triagem Urgente': round(percentual_sla_triagem_urgente_engenharia, 2) if sla_triagem_urgente_total_engenharia > 0 else 0,
    'ENG - Urgente Fechadas no Prazo': sla_fechamento_urgente_regular_total_engenharia,
    'ENG - % SLA Fechamento Urgente': round(percentual_sla_fechamento_urgente_regular_total_engenharia, 2) if sla_triagem_urgente_total_engenharia > 0 else 0,
    
    # Engenharia Clínica - SLA Pouco Urgente
    'ENG - OS Pouco Urgentes': sla_triagem_poucourgente_engenharia_total,
    'ENG - Pouco Urgente Atendidas no Prazo': sla_triagem_poucourgente_regular_engenharia_total,
    'ENG - % SLA Triagem Pouco Urgente': round(percentual_sla_poucourgente_engenharia, 2) if sla_triagem_poucourgente_engenharia_total > 0 else 0,
    'ENG - Pouco Urgente Fechadas no Prazo': sla_fechamento_poucourgente_regular_total_engenharia,
    'ENG - % SLA Fechamento Pouco Urgente': round(percentual_sla_fechamento_poucourgente_regular_total_engenharia, 2) if sla_triagem_poucourgente_engenharia_total > 0 else 0,
    
    # Engenharia Clínica - Distribuição Prioridades
    'ENG - % Emergentes': round(Percentagem_emergente, 2),
    'ENG - % Urgentes': round(percentagem_urgente, 2),
    'ENG - % Pouco Urgentes': round(percentagem_poucourgente, 2),
    
    # Predial - Corretivas
    'PRED - Total Corretivas Abertas': total_corretivas_criadas_predial,
    'PRED - Total Corretivas Fechadas': total_corretivas_predial_fechadas,
    'PRED - % Corretivas Fechadas': round(percentual_corretivas_fechadas_predial, 2),
    
    # Predial - Preventivas
    'PRED - Total Preventivas Abertas': total_preventivas_geradas_predial,
    'PRED - Total Preventivas Fechadas': total_preventivas_predial_fechada,
    'PRED - % Preventivas Fechadas': round(percentual_preventivas_fechadas_predial, 2),
    'PRED - Preventivas AR Abertas': total_preventivas_arcondicionado,
    'PRED - Preventivas AR Fechadas': total_preventivas_arcondicionado_fechadas,
    'PRED - Demais Preventivas Abertas': total_preventivas_demais,
    'PRED - Demais Preventivas Fechadas': total_preventivas_demais_fechadas,
    
    # Predial - SLA Emergente
    'PRED - OS Emergentes': sla_triagem_emergente_total_predial,
    'PRED - Emergente Atendidas no Prazo': sla_triagem_emergente_regular_total_predial,
    'PRED - % SLA Triagem Emergente': round(percentual_sla_triagem_emergente_predial, 2) if sla_triagem_emergente_regular_total_predial > 0 else 0,
    'PRED - Emergente Fechadas no Prazo': sla_fechamento_emergente_regular_total_predial,
    'PRED - % SLA Fechamento Emergente': round(percentual_sla_fechamento_emergente_predial, 2) if sla_fechamento_emergente_regular_total_predial > 0 else 0,
    
    # Predial - SLA Urgente
    'PRED - OS Urgentes': sla_triagem_urgente_predial_total,
    'PRED - Urgente Atendidas no Prazo': sla_triagem_urgente_regular_total_predial,
    'PRED - % SLA Triagem Urgente': round(percentual_sla_triagem_urgente_predial, 2),
    'PRED - Urgente Fechadas no Prazo': sla_fechamento_urgente_regular_total_predial,
    'PRED - % SLA Fechamento Urgente': round(percentual_sla_fechamento_urgente_regular_total_predial, 2),
    
    # Predial - SLA Pouco Urgente
    'PRED - OS Pouco Urgentes': sla_triagem_poucourgente_predial_total,
    'PRED - Pouco Urgente Atendidas no Prazo': sla_triagem_poucourgente_regular_predial_total,
    'PRED - % SLA Triagem Pouco Urgente': round(percentual_sla_poucourgente_predial, 2),
    'PRED - Pouco Urgente Fechadas no Prazo': sla_fechamento_poucourgente_regular_total_predial,
    'PRED - % SLA Fechamento Pouco Urgente': round(percentual_sla_fechamento_poucourgente_regular_total_predial, 2),
}

print(f"Total de indicadores coletados: {len(indicadores) - 1}")

Total de indicadores coletados: 55


In [54]:
# Criar DataFrame com uma linha (representando o mês atual)
df_indicadores = pd.DataFrame([indicadores])

# Exibir preview
print("Preview dos indicadores:")
print(df_indicadores.head())

Preview dos indicadores:
       Mês  Total Corretivas Abertas  Total Corretivas Fechadas  Total Preventivas Abertas  Total Preventivas Fechadas  ENG - Total Corretivas Abertas  ENG - Total Corretivas Fechadas  ENG - % Corretivas Fechadas  ENG - Corretivas Atendidas no Prazo  ENG - % Corretivas Atendidas no Prazo  ENG - Total Preventivas Abertas  ENG - Total Preventivas Fechadas  ENG - % Preventivas Fechadas  ENG - OS Emergentes  ENG - Emergente Atendidas no Prazo  ENG - % SLA Triagem Emergente  ENG - Emergente Fechadas no Prazo  ENG - % SLA Fechamento Emergente  ENG - OS Urgentes  ENG - Urgente Atendidas no Prazo  ENG - % SLA Triagem Urgente  ENG - Urgente Fechadas no Prazo  ENG - % SLA Fechamento Urgente  ENG - OS Pouco Urgentes  ENG - Pouco Urgente Atendidas no Prazo  ENG - % SLA Triagem Pouco Urgente  ENG - Pouco Urgente Fechadas no Prazo  ENG - % SLA Fechamento Pouco Urgente  ENG - % Emergentes  ENG - % Urgentes  ENG - % Pouco Urgentes  PRED - Total Corretivas Abertas  PRED - Total

In [55]:
# Caminho do arquivo CSV consolidado
arquivo_csv_consolidado = 'C:/Users/rafae/Documents/DSH/Clientes/COMG/Indicadores/indicadores_consolidados.csv'

# Verificar se o arquivo já existe
if os.path.exists(arquivo_csv_consolidado):
    # Ler arquivo existente
    df_existente = pd.read_csv(arquivo_csv_consolidado)
    
    # Verificar se o mês já existe
    if mes in df_existente['Mês'].values:
        # Atualizar a linha do mês existente
        df_existente = df_existente[df_existente['Mês'] != mes]
        df_final = pd.concat([df_existente, df_indicadores], ignore_index=True)
        print(f"Mês '{mes}' atualizado no arquivo existente.")
    else:
        # Adicionar nova linha
        df_final = pd.concat([df_existente, df_indicadores], ignore_index=True)
        print(f"Mês '{mes}' adicionado ao arquivo existente.")
else:
    # Criar novo arquivo
    df_final = df_indicadores
    print(f"Novo arquivo criado com o mês '{mes}'.")

# Ordenar por mês (opcional - você pode criar uma ordem customizada)
meses_ordem = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 
               'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
df_final['Ordem'] = df_final['Mês'].apply(lambda x: meses_ordem.index(x) if x in meses_ordem else 999)
df_final = df_final.sort_values('Ordem').drop('Ordem', axis=1)

# Salvar no CSV
df_final.to_csv(arquivo_csv_consolidado, index=False, encoding='utf-8-sig')
print(f"\nArquivo salvo com sucesso em: {arquivo_csv_consolidado}")
print(f"Total de meses no arquivo: {len(df_final)}")

Novo arquivo criado com o mês 'Janeiro'.


OSError: Cannot save file into a non-existent directory: 'C:\Users\rafae\Documents\DSH\Clientes\COMG\Indicadores'

***Visualizar resumo do arquivo consolidado***

In [ ]:
# Exibir as primeiras colunas do arquivo consolidado
print("Resumo do arquivo consolidado:")
print(f"Dimensões: {df_final.shape[0]} meses x {df_final.shape[1]} indicadores")
print("\nPrimeiras colunas:")
print(df_final.iloc[:, :10].to_string())

Resumo do arquivo consolidado:
Dimensões: 1 meses x 56 indicadores

Primeiras colunas:
       Mês  Total Corretivas Abertas  Total Corretivas Fechadas  Total Preventivas Abertas  Total Preventivas Fechadas  ENG - Total Corretivas Abertas  ENG - Total Corretivas Fechadas  ENG - % Corretivas Fechadas  ENG - Corretivas Atendidas no Prazo  ENG - % Corretivas Atendidas no Prazo
0  Janeiro                       220                        217                        452                         299                              72                               72                        100.0                                   53                                  73.61
